In [11]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [12]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [13]:
df = pd.read_sql('SELECT * FROM marketsmojorecos', con=database_connection)

In [14]:
start_date = '2017-01-01'
end_date = '2017-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [15]:
mask = (df['predict_date'] > date_object1) & (df['predict_date'] <= date_object2)

# 2017 data

In [16]:
df1 = df.loc[mask]
df1 = df1.reset_index()
del df1['level_0']
del df1['srno']
del df1['index']

In [17]:
print("No of Brokers in 2017 :",df1['broker'].nunique()) 
s1=len(df1)
df1

No of Brokers in 2017 : 62


,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Indiabulls Ventures,L&T Finance Holdings,91.80,163.54,2017-08-11,230.00
1,Motilal Oswal,Tech Mahindra,696.90,388.00,2017-06-29,465.00
2,Ventura,APL Apollo Tubes,1278.85,1525.00,2017-06-27,2647.6
3,ICICIdirect.com,Power Grid Corpn.,199.65,208.00,2017-06-01,230.00
4,Edelweiss,SKF India,1881.40,1565.00,2017-05-30,1716.00
...,...,...,...,...,...,...
10280,ICICIdirect.com,GAIL India,129.90,187.13,2017-01-03,213.58
10281,ICICIdirect.com,Glaxo.Cons. Health,7881.30,5030.00,2017-01-03,6765.00
10282,Motilal Oswal,PVR,1482.00,1167.00,2017-01-03,1429.00
10283,ICICIdirect.com,KEC International,245.30,142.4,2017-01-03,170.00


In [18]:
df3=pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'company_name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sqlrun = "SELECT SYMBOL FROM symbolsmapped WHERE recoName = '"+sym+"'"
    df3 = df3.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

In [19]:
df4 =pd.concat([df1,df3],axis=1)
df4

,broker,company_name,current_price,recomended_buying,predict_date,target_price,SYMBOL
0,Indiabulls Ventures,L&T Finance Holdings,91.80,163.54,2017-08-11,230.00,L&TFH
1,Motilal Oswal,Tech Mahindra,696.90,388.00,2017-06-29,465.00,TECHM
2,Ventura,APL Apollo Tubes,1278.85,1525.00,2017-06-27,2647.6,APLAPOLLO
3,ICICIdirect.com,Power Grid Corpn.,199.65,208.00,2017-06-01,230.00,POWERGRID
4,Edelweiss,SKF India,1881.40,1565.00,2017-05-30,1716.00,SKFINDIA
...,...,...,...,...,...,...,...
10280,ICICIdirect.com,GAIL India,129.90,187.13,2017-01-03,213.58,GAIL
10281,ICICIdirect.com,Glaxo.Cons. Health,7881.30,5030.00,2017-01-03,6765.00,GSKCONS
10282,Motilal Oswal,PVR,1482.00,1167.00,2017-01-03,1429.00,PVR
10283,ICICIdirect.com,KEC International,245.30,142.4,2017-01-03,170.00,KEC


In [20]:
#dftest = df4[0:100]
#dftest

In [21]:
uniqueBrokers = df4.broker.unique()
len(uniqueBrokers)

62

In [24]:
#del brokerPerformace
brokerPerformace = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
#brokerPerformace = brokerPerformace.append({'Name':'a','Hit Ratio':'b','Avg. Growth':'c'}, ignore_index=True)
brokerPerformace

,Name,Hit Ratio,Avg. Growth,Total Recos


In [25]:
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['broker'] == brokerName]
    #print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        #print(type(sym))
        if type(sym) == str :
            mydate1 = row['predict_date']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
        
            sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
            del dfprices1['index']
        
            sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
            dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
            del dfprices2['index']
            dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)

            targetp = float(row['target_price'])
        
            #print("SYMBOL : ",sym)
            dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
            total = total + 1
            
            if not dfhits.empty :
                hits = hits + 1
            
            recbuying = float(row['recomended_buying'])
            maxval = dfprices['Closeval'].max()
            growthperc = (maxval - recbuying) *100 / recbuying
            #print('Growth : ',growthperc)
            if np.isnan(growthperc):
                total = total-1
            else:    
                growthSum = growthSum + growthperc 
    if total != 0:
        hitRatio = hits / total
        avgGrowth = growthSum / total
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformace = brokerPerformace.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Broker :  Indiabulls Ventures HitRatio :  0.3333333333333333 Avg. Growth :  79.21334132130958 Total Recos. :  21
Broker :  Motilal Oswal HitRatio :  0.6702127659574468 Avg. Growth :  107.87686741090418 Total Recos. :  1222
Broker :  Ventura HitRatio :  0.43478260869565216 Avg. Growth :  77.12776303916976 Total Recos. :  23


KeyboardInterrupt: 

In [69]:
brokerPerformace

,Name,Hit Ratio,Avg. Growth,Total Recos
0,KRChoksey,0.693780,83.563415,209
1,AUM Capital,0.714286,69.171851,21
2,Religare,0.803867,61.607401,362
3,Enochventures.com,0.433333,42.221521,30
4,Edelweiss,0.756303,147.177093,476
5,Motilal Oswal,0.701172,145.850515,512
6,Kotak Securities,0.821705,672.562100,129
7,Angel Broking,0.575472,77.827408,106
8,Geojit Financial Services,0.831731,273.519232,208
9,Centrum,0.733906,51.254127,233


In [71]:
brokerPerformace.to_sql(con=database_connection, name='brokerperformance2016', if_exists='append')